In this tutorial, you'll learn how to execute a QM manager and monitor it.

In [1]:
import os
import dotenv

from tqdm.auto import tqdm

import pandas as pd

from qcportal import PortalClient

_ = dotenv.load_dotenv("../../openfractal_test_secrets.env")

## Launch a manager

A manager is a QM worker that will perform any QM calculations provided by an Openfractal instance.

You first need to create a YAML config file `manager_test.yml`:

```yaml
# The base folder to use as the default for some options (logs, etc).
base_folder: /tmp/qcf_compute

# The name of the manager is used to identify the manager in the database.
# It's not unique but for each manager launched a unique name will be computed
# by the server.
cluster: manager_demo_local_1

loglevel: INFO
logfile: null

# Time between heartbeats/update checks between this Manager and the Fractal Server.
update_frequency: 30

# Settings to connect to the Fractal Server.
server:
  fractal_uri: https://openfractal-test-pgzbs3yryq-uc.a.run.app
  username: YOUR_USERNAME
  password: YOUR_PASSWORD
  verify: false

# How and where to detect the QM softwares.
environments:
  use_manager_environment: true
  conda: []
  apptainer: []

executors:
  local:
    type: local

    # Common to all executors.

    # Tags are used to filter the tasks that will be sent to the manager.
    queue_tags: ["demo"]
    worker_init: []
    scratch_directory: null
    bind_address: null
    cores_per_worker: 16
    memory_per_worker: 16 # GB
    extra_executor_options: {}

    # Specific options for the local executor.
    max_workers: 4

```

Then you can start a manager with:

```bash
qcfractal-compute-manager --config manager_test.yml
```

## Monitor the managers

In [2]:
client = PortalClient(
    address="https://openfractal-test-pgzbs3yryq-uc.a.run.app",
    username=os.environ["OPENFRACTAL_USER_5_USERNAME"],
    password=os.environ["OPENFRACTAL_USER_5_PASSWORD"],
)

client

PortalClient(server_name='openfractal-test', address='https://openfractal-test-pgzbs3yryq-uc.a.run.app/', username='monitor_default')

In [3]:
# Check connected compute managers (workers)
managers = pd.DataFrame([m.dict() for m in client.query_managers()])
managers

,id,name,cluster,hostname,username,tags,claimed,successes,failures,rejected,total_cpu_hours,active_tasks,active_cores,active_memory,status,created_on,modified_on,manager_version,programs,log_
0,3,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,manager_demo_local,boromir,admin_default,[demo_local],10,8,2,0,0.251861,1,8,16.0,ManagerStatusEnum.inactive,2023-06-12 20:22:58.917570,2023-06-12 20:28:28.753931,0.50b11.post14+gae51d47c,"{'qcengine': ['v0.26.0'], 'rdkit': ['unknown']...",None
1,2,manager_demo_local-boromir-c4f22a20-1f48-447f-...,manager_demo_local,boromir,admin_default,[demo_local],0,0,0,0,0.000000,1,8,16.0,ManagerStatusEnum.inactive,2023-06-12 20:20:26.453910,2023-06-12 20:22:03.060516,0.50b11.post14+gae51d47c,"{'qcengine': ['v0.26.0'], 'rdkit': ['unknown']...",None
2,1,manager_demo_local-boromir-c1a9b892-b0b7-49f9-...,manager_demo_local,boromir,admin_default,[demo_local],0,0,0,0,0.000000,1,8,16.0,ManagerStatusEnum.inactive,2023-06-12 20:08:33.378386,2023-06-12 20:09:42.659622,0.50b11.post14+gae51d47c,"{'qcengine': ['v0.26.0'], 'psi4': ['unknown'],...",None


## Monitor your dataset

In [15]:
dataset_name = "dataset_demo_5077749542"

ds = client.get_dataset("singlepoint", dataset_name)
ds

SinglepointDataset(id=4, dataset_type='singlepoint', name='dataset_demo_5077749542', description='my great dataset!', tagline='', tags=['demo_local'], group='default', visibility=True, provenance={}, default_tag='demo_local', default_priority=<PriorityEnum.normal: 1>, owner_user='admin_default', owner_group=None, metadata={}, extras={}, entry_names_=[], specifications_={}, entries_={}, record_map_={}, contributed_values_=None, auto_fetch_missing=True)

Refresh the below often.

In [20]:
print(ds.status_table())

             specification    complete    running
--------------------------  ----------  ---------
simple_qm_calculation_demo           3          7


In [24]:
records_list = []
for r in tqdm(client.query_records(dataset_id=ds.id)):
    # Access those objects to fetch them locally
    r.error
    r.wavefunction
    records_list.append(r.dict())

records = pd.DataFrame(records_list)
records = records.sort_values("id")
records = records.reset_index(drop=True)

records

0it [00:00, ?it/s]

,id,record_type,is_service,properties,extras,status,manager_name,created_on,modified_on,owner_user,owner_group,compute_history_,task_,service_,comments_,native_files_,specification,molecule_id,molecule_,wavefunction_
0,11,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [0.0335809823...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161042,2023-06-12 20:23:29.625597,admin_default,None,"[{'id': 1, 'record_id': 11, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",28,None,"{'compression_type': 'CompressionEnum.zstd', '..."
1,12,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [-0.191476353...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161047,2023-06-12 20:23:29.788551,admin_default,None,"[{'id': 2, 'record_id': 12, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",21,None,"{'compression_type': 'CompressionEnum.zstd', '..."
2,13,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [0.0032987900...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161048,2023-06-12 20:23:29.856461,admin_default,None,"[{'id': 3, 'record_id': 13, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",24,None,"{'compression_type': 'CompressionEnum.zstd', '..."
3,14,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [-0.356400390...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161049,2023-06-12 20:24:00.283171,admin_default,None,"[{'id': 4, 'record_id': 14, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",20,None,"{'compression_type': 'CompressionEnum.zstd', '..."
4,15,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [0.0085374704...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161050,2023-06-12 20:24:00.373625,admin_default,None,"[{'id': 5, 'record_id': 15, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",27,None,"{'compression_type': 'CompressionEnum.zstd', '..."
5,16,singlepoint,False,"{'pe energy': 0.0, 'scf dipole': [0.8039027310...",{},RecordStatusEnum.complete,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161051,2023-06-12 20:24:30.943935,admin_default,None,"[{'id': 6, 'record_id': 16, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",23,None,"{'compression_type': 'CompressionEnum.zstd', '..."
6,17,singlepoint,False,None,None,RecordStatusEnum.error,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161052,2023-06-12 20:24:31.153263,admin_default,None,"[{'id': 7, 'record_id': 17, 'status': 'RecordS...",None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",26,None,None
7,18,singlepoint,False,None,None,RecordStatusEnum.running,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161053,2023-06-12 20:22:59.001990,admin_default,None,[],None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",29,None,None
8,19,singlepoint,False,None,None,RecordStatusEnum.running,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161054,2023-06-12 20:22:59.001996,admin_default,None,[],None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",25,None,None
9,20,singlepoint,False,None,None,RecordStatusEnum.running,manager_demo_local-boromir-86f6c0ce-e825-41f3-...,2023-06-12 20:22:35.161054,2023-06-12 20:23:30.024544,admin_default,None,[],None,None,None,None,"{'program': 'psi4', 'driver': 'SinglepointDriv...",22,None,None


In [27]:
row = records.iloc[0]

row["properties"].keys()

dict_keys(['pe energy', 'scf dipole', 'calcinfo_nmo', 'mbis charges', 'mbis dipoles', 'mayer indices', 'mayer_indices', 'return_energy', 'return_result', 'calcinfo_natom', 'calcinfo_nbeta', 'current dipole', 'current energy', 'lowdin charges', 'lowdin_charges', 'mbis octupoles', 'return_hessian', 'scf iterations', 'scf quadrupole', 'scf_iterations', 'calcinfo_nalpha', 'calcinfo_nbasis', 'hf total energy', 'hf virial ratio', 'return_gradient', 'current gradient', 'mbis quadrupoles', 'scf total energy', 'scf_total_energy', 'hf kinetic energy', 'hf total gradient', 'scf_dipole_moment', 'scf_total_hessian', 'scf total gradient', 'scf_total_gradient', 'dd solvation energy', 'hf potential energy', 'mbis valence widths', 'one-electron energy', 'two-electron energy', 'scf iteration energy', 'wiberg lowdin indices', 'wiberg_lowdin_indices', 'pcm polarization energy', 'scf_one_electron_energy', 'scf_two_electron_energy', 'current reference energy', 'nuclear repulsion energy', 'nuclear_repulsion_

In [29]:
row["wavefunction_"].keys()

dict_keys(['compression_type', 'data_url_', 'compressed_data_', 'decompressed_data_'])